-   Diabetec data set model

In [37]:
import numpy as np
import pandas as pd
import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [72]:
class dataRead(object):
    def __init__(self,file):
        self.file=file
    def load_data(self):
        data=pd.read_csv(self.file,header=None,skiprows=1,dtype=np.float32)
        data1=data.copy(deep=True)
        feature=data1.iloc[:,0:8]
        target=data1[8]
        return feature,target
    
    def scaling(self,X_train):
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train_minmax = min_max_scaler.fit_transform(X_train)
        return X_train_minmax
            
    
    def split(self,X,y):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
        return X_train, X_test, y_train, y_test
   
        
    

In [74]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

### Train model 

In [75]:

def trainModel(trainX, trainY,modreference):
    use_cuda = False
    FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
    LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
    Tensor = FloatTensor 
        
    # fix seed
    seed=17*19
    np.random.seed(seed)
    
    # Train data
    x_data_np = np.array(trainX, dtype=np.float32)
    y_data_np = np.array(trainY, dtype=np.float32)
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!
        
    print(x_data_np.shape, y_data_np.shape)
    print(type(x_data_np), type(y_data_np))
    #if use_cuda:
               
       # X = Variable(torch.from_numpy(x_data_np).cuda()) # Note the conversion for pytorch
        #Y = Variable(torch.from_numpy(y_data_np).cuda())
    #else:
            
    X = Variable(torch.from_numpy(x_data_np)) # Note the conversion for pytorch
    Y = Variable(torch.from_numpy(y_data_np))    
    print(type(X.data), type(Y.data)) # should be 'torch.cuda.FloatTensor'
    
        #print(type(X.data), type(Y.data)) # should be 'torch.cuda.FloatTensor'

        
    # our model
    model = modreference


    # Construct our loss function and an Optimizer. The call to model.parameters()
    # in the SGD constructor will contain the learnable parameters of the two
    # nn.Linear modules which are members of the model.
    criterion = torch.nn.BCELoss(size_average=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    
    # Training loop
    for epoch in range(100):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(X)

        # Compute and print loss
        loss = criterion(y_pred, Y)
        print(epoch, loss.data[0])

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        
    
       
                  
                  


In [79]:
def main():
    file='E:/DataSet/diabetesData/diabetes.csv'
    obj=dataRead(file)
    feature,target=obj.load_data()
    scaleData=obj.scaling(feature)
    X_train, X_test, y_train, y_test=obj.split(scaleData,target)
    #print(X_train.head())
    model=Model()
    trainModel(X_train, y_train,model)
    
    #print(scaleData)
    

In [80]:
main()

(514, 8) (514, 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
0 tensor(0.6570)
1 tensor(0.6563)
2 tensor(0.6558)
3 tensor(0.6552)
4 tensor(0.6548)
5 tensor(0.6544)
6 tensor(0.6540)
7 tensor(0.6536)
8 tensor(0.6533)
9 tensor(0.6530)
10 tensor(0.6528)
11 tensor(0.6525)
12 tensor(0.6523)
13 tensor(0.6521)
14 tensor(0.6519)
15 tensor(0.6518)
16 tensor(0.6516)
17 tensor(0.6515)
18 tensor(0.6514)
19 tensor(0.6513)
20 tensor(0.6512)
21 tensor(0.6511)
22 tensor(0.6510)
23 tensor(0.6509)
24 tensor(0.6508)
25 tensor(0.6508)
26 tensor(0.6507)
27 tensor(0.6507)
28 tensor(0.6506)
29 tensor(0.6506)
30 tensor(0.6505)
31 tensor(0.6505)
32 tensor(0.6505)
33 tensor(0.6504)
34 tensor(0.6504)
35 tensor(0.6504)
36 tensor(0.6504)
37 tensor(0.6503)
38 tensor(0.6503)
39 tensor(0.6503)
40 tensor(0.6503)
41 tensor(0.6503)
42 tensor(0.6503)
43 tensor(0.6503)
44 tensor(0.6502)
45 tensor(0.6502)
46 tensor(0.6502)
47 tensor(0.6502)
48 tensor(0.6502)
49 tensor(0.6502

C:\Users\admin\Anaconda3\envs\pyt\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\admin\Anaconda3\envs\pyt\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))
C:\Users\admin\Anaconda3\envs\pyt\lib\site-packages\ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
